In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


In [ ]:
df = pd.read_csv("results/daily_sleep_summary.csv")

if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'])

print("✅ Data loaded successfully!")
print(df.head())


In [ ]:
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

if not numeric_cols:
    raise ValueError("No numeric columns found in the dataset!")

df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())


In [ ]:
iso = IsolationForest(contamination=0.1, random_state=42)
df["anomaly"] = iso.fit_predict(df[numeric_cols])
df["is_anomaly"] = df["anomaly"] == -1

print(f"🚨 Detected {df['is_anomaly'].sum()} anomalies out of {len(df)} total days.")


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df["date"], df["total_sleep_hours"], label="Total Sleep (hours)", marker='o')
plt.scatter(
    df.loc[df["is_anomaly"], "date"],
    df.loc[df["is_anomaly"], "total_sleep_hours"],
    color="red", label="Anomaly", s=80
)
plt.title("🧠 Sleep Pattern Anomalies")
plt.xlabel("Date")
plt.ylabel("Total Sleep (hours)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
anomalies = df.loc[df["is_anomaly"], ["date", "total_sleep_hours"] + [c for c in numeric_cols if c != "total_sleep_hours"]]

sample_data = anomalies.head(10).to_markdown(index=False)

prompt = f"""
You are a sleep scientist analyzing daily sleep data anomalies.

Below is a sample of detected unusual days in the user's sleep metrics:

{sample_data}

Explain:
- What kind of irregularities are visible (short/long sleep, heart rate spikes, etc.)
- Possible causes (stress, exercise change, inconsistent bedtime)
- Actionable suggestions to restore healthy sleep patterns

Keep it analytical yet easy to understand.
"""


In [ ]:
response = client.responses.create(
    model=os.getenv("GPT_MODEL"),
    input=prompt,
    temperature=0.7,
)

print("🧠 Fine-Tuned AI Anomaly Analysis:\n")
print(response.output[0].content[0].text)
